In [3]:
from random import randint
import random
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:2][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)

for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,576


In [9]:
all_results = []
total_simple = 0
total_average = 0
total_complex = 0
for i in range(100):
    #print('try ', i)
    mae = 0
    final_results = []
    for index, row in col1.iterrows():
        #print()
        #print("____ DATA ",index,"____")

        swarm_size = 70
        pf = 20
        max_iter = 50
        c1 = 2
        c2 = 2
        posisi = {}
        partikel = {}
        fitness_value = 200

        rentangposisi = {
            "simple": {
                "lower": 5,
                "upper": 7.49
            },
            "average": {
                "lower": 7.5,
                "upper": 12.49
            },
            "complex": {
                "lower": 12.5,
                "upper": 15
            }
        }

        projectactual = {
            'simpleUC': col1.loc[index,"S"],
            'averageUC': col1.loc[index,"A"],
            'complexUC': col1.loc[index,"C"],
            'uaw': col1.loc[index,"UAW"],
            'tcf': col1.loc[index,"TCF"],
            'ecf': col1.loc[index,"ECF"],
            'actualEffort': col1.loc[index,"Real_Effort"]
        }


        def posisiacak():
            xsimple = random.uniform(rentangposisi['simple']['lower'], rentangposisi['simple']['upper'])
            xaverage = random.uniform(rentangposisi['average']['lower'], rentangposisi['average']['upper'])
            xcomplex = random.uniform(rentangposisi['complex']['lower'], rentangposisi['complex']['upper'])
            position = {
                        'simple':xsimple,
                        'average':xaverage,
                        'complex':xcomplex
                        }   
            return position

        def size(xsimple, xaverage, xcomplex):
            ucsimple = xsimple * projectactual['simpleUC']
            ucaverage = xaverage * projectactual['averageUC']
            uccomplex = xcomplex * projectactual['complexUC']
            uucw = ucsimple + ucaverage + uccomplex
            uucp = projectactual['uaw'] + uucw
            return uucp * projectactual['tcf'] * projectactual['ecf']

        def velocity(parameters):
            vsimple = (parameters['w'] * parameters['partikel']['vsimple']) + ((c1 * parameters['r1']) * (parameters['pbest']['simple'] - parameters['partikel']['simple'])) + ((c2 * parameters['r2']) * (parameters['gbest']['simple'] - parameters['partikel']['simple']))

            vaverage = (parameters['w'] * parameters['partikel']['vaverage']) + ((c1 * parameters['r1']) * (parameters['pbest']['average'] - parameters['partikel']['average'])) + ((c2 * parameters['r2']) * (parameters['gbest']['average'] - parameters['partikel']['average']))

            vcomplex = (parameters['w'] * parameters['partikel']['vcomplex']) + ((c1 * parameters['r1']) * (parameters['pbest']['complex'] - parameters['partikel']['complex'])) + ((c2 * parameters['r2']) * (parameters['gbest']['complex'] - parameters['partikel']['complex']))

            return {'vsimple': vsimple, 'vaverage': vaverage, 'vcomplex': vcomplex}

        def inertia(iterasi):
            inertia_max = 0.9
            inertia_min = 0.4
            return inertia_min - (((inertia_max - inertia_min) * iterasi) / max_iter)

        def exceedlimit(value, label):
            if label == 'simple' and value < rentangposisi['simple']['lower']:
                value = rentangposisi['simple']['lower']
            if label == 'simple' and value > rentangposisi['simple']['upper']:
                value = rentangposisi['simple']['upper']
            if label == 'average' and value < rentangposisi['average']['lower']:
                value = rentangposisi['average']['lower']
            if label == 'average' and value > rentangposisi['average']['upper']:
                value = rentangposisi['average']['upper']
            if label == 'complex' and value < rentangposisi['complex']['lower']:
                value = rentangposisi['complex']['lower']
            if label == 'complex' and value > rentangposisi['complex']['upper']:
                value = rentangposisi['complex']['upper']
            return value

        def optimizer():
            partikel = []
            gbests = []
            pbests = []
            ret = []
            for iter in range(max_iter):
                #print()
                #print('iterasi', iter)
                w = inertia(iter)
                r1 = random.uniform(0, 1)
                r2 = random.uniform(0, 1)

                # initial population
                if iter == 0: 
                    for i in range(swarm_size):
                        xsimple = posisiacak()['simple']
                        xaverage = posisiacak()['average']
                        xcomplex = posisiacak()['complex']
                        software_size = size(xsimple,xaverage,xcomplex)
                        estimated_effort = software_size * pf
                        ae = abs(projectactual['actualEffort'] - estimated_effort)
                        vsimple = random.uniform(0, 1)
                        vaverage = random.uniform(0, 1)
                        vcomplex = random.uniform(0, 1)
                        particles = {
                            'simple': xsimple,
                            'average': xaverage,
                            'complex': xcomplex,
                            'size': software_size,
                            'estimated': estimated_effort,
                            'ae': ae,
                            'vsimple':vsimple,
                            'vaverage':vaverage,
                            'vcomplex':vcomplex
                        }
                        partikel.append([])
                        partikel[iter].append(particles)
                        pbests.append([])
                        pbests[iter].append(particles)
                    min_ae = min(x['ae'] for x in pbests[iter])
                    gbest = next(item for item in pbests[iter] if item['ae'] == min_ae)
                    gbests.append([])
                    gbests[iter].append(gbest)
                    #print(gbests[iter])

                if iter > 0:
                    for count,value in enumerate(partikel[iter-1]):
                        parameters = {
                            'pbest': pbests[iter-1][count],
                            'partikel': value,
                            'gbest': gbests[iter-1][0],
                            'w':w,
                            'r1':r1,
                            'r2':r2
                        }
                        vel = velocity(parameters)
                        xsimple = value['simple'] + vel['vsimple']
                        xaverage = value['average'] + vel['vaverage']
                        xcomplex = value['complex'] + vel['vcomplex']
                        xsimple = exceedlimit(xsimple, 'simple')
                        xaverage = exceedlimit(xaverage, 'average')
                        xcomplex = exceedlimit(xcomplex, 'complex')
                        software_size = size(xsimple,xaverage,xcomplex)
                        estimated_effort = software_size * pf
                        ae = abs(projectactual['actualEffort'] - estimated_effort)
                        particles = {
                            'simple': xsimple,
                            'average': xaverage,
                            'complex': xcomplex,
                            'size': software_size,
                            'estimated': estimated_effort,
                            'ae': ae,
                            'vsimple':vel['vsimple'],
                            'vaverage':vel['vaverage'],
                            'vcomplex':vel['vcomplex']
                        }
                        partikel.append([])
                        partikel[iter].append(particles)

                        if pbests[iter-1][count]['ae'] > partikel[iter][count]['ae']:
                           pbests[iter].append(partikel[iter][count])
                        else:
                            pbests[iter].append(partikel[iter-1][count])

                    min_ae = min(x['ae'] for x in pbests[iter])
                    gbest = next(item for item in pbests[iter] if item['ae'] == min_ae)
                    gbests.append([])
                    gbests[iter].append(gbest)
                    #print(gbests[iter])

                    if gbests[iter][0]['ae'] < fitness_value:
                        return gbests[iter][0]
                    ret.append(gbests[iter][0])
            return ret

        results = optimizer()

        #print(projectactual)
        #print()
        #print('results ',results)    
        #print()
        
        if len(results) == 9:
            mae = mae + results['ae']
            final_results.append(results)
                    
        else:
            min_ae = min(x['ae'] for x in results)
            results = next(item for item in results if item['ae'] == min_ae)
            mae = mae + results['ae']
            final_results.append(results)
        
    all_results.append(final_results)
    
    #print()
    #print()
    #print('mae', i, mae/71)

for x in (all_results):
    #for y in (x):
        #print(y['simple'])
    #print(all_results[x][1])
    print(x[0]['simple'])
    total_simple = total_simple + x[0]['simple']
    #total_average = total_simple + x[0]['average']
    #total_complex = total_complex + x[0]['complex']

print()
print('total simple', total_simple/100)
#print('total average', total_average/50)
#print('total complex', total_complex/50)
    

    
#print(all_results, len(all_results))
#print('all_results')
#print(all_results)

7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.071423890309489
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49
7.49

total simple 7.485814238903101
